Paper: "Women’s Preferences for Treatment of Perinatal Depression and Anxiety: A Discrete Choice Experiment"

Author(s): Jemimah Ride, Emily Lancsar

Year: 2016

Model(s): Multinomial Logit, Mixed Logit

Main findings: \_\_\_\_

# Import needed libraries

In [1]:
import sys
from collections import OrderedDict

import scipy.stats

import numpy as np
import pandas as pd

import seaborn as sbn
import matplotlib.pyplot as plt

from statsmodels.formula.api import logit

sys.path.insert(0, '/Users/timothyb0912/Documents/pylogit/')
import pylogit as pl

sys.path.insert(0, '../../src/')
from visualization import predictive_viz as viz

%matplotlib inline

/Users/timothyb0912/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Import the model-ready data

In [2]:
clean_data_path =\
    '../../data/processed/ride_2016/ride_2016_final_data.csv'
clean_df = pd.read_csv(clean_data_path)

# Look at data

In [10]:
# Look at the shape of clean_df
print('clean_df.shape == {}'.format(clean_df.shape))

# Look at some records in clean_df
clean_df.head(6).T


clean_df.shape == (10416, 51)


,41,43,44,14,18,45
choice,1,0,0,0,1,0
alt,1,2,3,1,2,3
id,101,101,101,101,101,101
cset,1,1,1,2,2,2
block,2,2,2,2,2,2
preg,Currently pregnant,Currently pregnant,Currently pregnant,Currently pregnant,Currently pregnant,Currently pregnant
post,Has not had baby in last 12 months,Has not had baby in last 12 months,Has not had baby in last 12 months,Has not had baby in last 12 months,Has not had baby in last 12 months,Has not had baby in last 12 months
lact,not currently breastfeeding,not currently breastfeeding,not currently breastfeeding,not currently breastfeeding,not currently breastfeeding,not currently breastfeeding
age,26,26,26,26,26,26
educ,postgrad,postgrad,postgrad,postgrad,postgrad,postgrad


In [5]:
set(clean_df.columns.tolist())

{'ASC',
 'IRDdecile',
 'MYE',
 'age',
 'alt',
 'block',
 'child',
 'children',
 'choice',
 'clinic',
 'combo',
 'cost',
 'couns',
 'country',
 'cset',
 'educ',
 'effect',
 'employ',
 'epc',
 'group',
 'hcc',
 'helpseek',
 'herb',
 'home',
 'id',
 'income',
 'lact',
 'lang',
 'married',
 'med',
 'online',
 'pasmbu',
 'pastcouns',
 'pastepc',
 'pastgrp',
 'pastherb',
 'pastinpt',
 'pastmedanx',
 'pastmeddep',
 'pastmedpsy',
 'pastnone',
 'pastpeer',
 'pastsleepmedication',
 'pastyoga',
 'peer',
 'phi',
 'phone',
 'post',
 'preg',
 'support'}

In [6]:
# How many individuals are in the dataset?
clean_df.id.unique().size

217

In [7]:
# How many choice sets per person?
clean_df.cset.unique().size

16

# Create needed objects for specification and estimation

In [8]:
# Give names to the alternatives
alt_id_map = {'A': 1, 'B': 2, 'N':3}
alt_id_reverse_map = {alt_id_map[k]: k for k in alt_id_map}

In [11]:
# Record the columns that denote the various treatment types
treatment_types = ['couns', 'combo', 'peer', 'group',
                   'epc', 'herb', 'MYE']
# Note the names of these variables as they should be displayed
# in the estimation results table
treatment_names =\
    ['Counseling',
     'Counseling & medication',
     'Peer support',
     'Group counseling',
     'Early parenting centre programme',
     'Natural, herbal or traditional Chinese medicine',
     'Meditation, yoga or exercise',
    ]

# Record the various consulatation types and the names that
# should be used for these types in the estimation results table.
consultation_types = ['home', 'phone', 'online']
consultation_names = ['Home visit', 'Telephone', 'Online']

# Record the interactions between varioius treatment types and 
# the names that should be used for these types in the
# estimation results table.
highschool_treatment_types =\
    [x + '_highschool' for x in treatment_types]
highschool_treatment_names =\
    [x + ' (Highschool)' for x in treatment_names]

breastfeeding_treatment_types =\
    [x + '_breastfeeding' for x in treatment_types]
breastfeeding_treatment_names =\
    [x + ' (Breastfeeding)' for x in treatment_names]

pregnant_treatment_types =\
    [x + '_pregnant' for x in treatment_types]
pregnant_treatment_names =\
    [x + ' (Pregnant)' for x in treatment_names]

# Map each treatment type to it's corresponding "past" variable
past_to_treatment_map =\
    {'couns': 'pastcouns',
     'combo': 'past_combo',
     'peer': 'pastpeer',
     'group': 'pastgrp',
     'epc': 'pastepc',
     'herb': 'pastherb',
     'MYE': 'pastyoga',
     'med': 'past_medication',
    }
past_treatment_cols = past_to_treatment_map.values()

# Define the model specification and parameter names

In [16]:
explanatory_vars =\
    (['ASC'] +
     treatment_types +
     ['cost',
      'effect'] +
     consultation_types +
     ['child',
      'Age',
      'Employed',
      'Unemployed',
      'past_experience_any_type',
      'low_support',
      'seek_help',
      'income_times_cost',
      'insurance_times_cost',
      'prior_experience_of_treatment_type',
     ] +
     highschool_treatment_types +
     breastfeeding_treatment_types +
     pregnant_treatment_types
    )

display_names =\
    (['Alternative Specific Constant'] +
     treatment_names +
     ['Medication cost',
      'Likely to improve'] +
     consultation_names +
     ['Childcare available',
      'Age',
      'Employed',
      'Unemployed',
      'Experience of any treatment type',
      'Low support levels',
      'States would seek help',
      'Income * Cost',
      'Private health insurance * Cost',
      'Experience of matched treatment type',
     ] +
     highschool_treatment_names +
     breastfeeding_treatment_names +
     pregnant_treatment_names
    )

missing_vars =\
    [x for x in explanatory_vars if x not in clean_df.columns]
if len(missing_vars) > 0:
    msg = 'These explanatory variables are not in the data file:'
    raise ValueError(msg + '\n{}'.format(missing_vars))

assert len(display_names) == len(explanatory_vars)
# Populate the specification and name dictionaries
class SpecInfo(object):
    def __init__(self,
                 variable_list,
                 display_names):
        self.variable_list = variable_list
        self.name_list = display_names
        self.spec_dict = OrderedDict()
        self.name_dict = OrderedDict()
        
        self.populate_spec_and_name_dicts()
        return None
        
    def populate_spec_and_name_dicts(self):
        num_vars = len(self.variable_list)
        for pos in range(num_vars):
            current_column = self.variable_list[pos]
            self.spec_dict[current_column] = 'all_same'
            self.name_dict[current_column] = self.name_list[pos]
        return None

base_model_info = SpecInfo(explanatory_vars, display_names)

# Set model parameters

In [20]:
# This column denotes whether the row's alternative was chosen
choice_col = 'choice'
# This column denotes the alternative corresponding to the row
alt_id_col = 'alt'
# This column denotes the id of the row's unique choice situation
obs_id_col = 'obs_id'
# This column denotes a particular decision maker
mixing_col = 'id'
# Determine the variables being mixed
mixing_variables =\
    (['Alternative Specific Constant'] +
     treatment_names +
     ['Medication cost',
      'Likely to improve'] +
     consultation_names +
     ['Childcare available'])
# Determine the number of draws for estimation
num_mxl_draws = 300

# Create the model object(s)

In [22]:
mnl_obj =\
    pl.create_choice_model(
        data=clean_df,
        alt_id_col=alt_id_col,
        obs_id_col=obs_id_col,
        choice_col=choice_col,
        specification=base_model_info.spec_dict,
        model_type='MNL',
        names=base_model_info.name_dict)

model_obj =\
    pl.create_choice_model(
        data=clean_df,
        alt_id_col=alt_id_col,
        obs_id_col=obs_id_col,
        choice_col=choice_col,
        specification=base_model_info.spec_dict,
        model_type='Mixed Logit',
        names=base_model_info.name_dict,
        mixing_id_col=mixing_col,
        mixing_vars=mixing_variables)

# Estimate and view the model

In [33]:
mnl_results['x'].tolist()

[1.6956860381997552,
 0.9650680119227433,
 1.465650058401747,
 0.6807409761037023,
 0.6963686049225445,
 1.3313163908388466,
 0.9830561450778008,
 1.4517300240710496,
 -0.014762984923495889,
 0.3867897028205267,
 -0.0588532936015507,
 -0.22040082974847408,
 -0.0850658869547287,
 0.22291936730296935,
 -0.03594211422241635,
 0.2286511502206337,
 -1.3775856880585402,
 0.61444715221664,
 -0.6367489309487726,
 0.3330385772971426,
 3.9487291659513576e-06,
 0.0032802344697110374,
 0.3505181625313443,
 -0.5963430815873182,
 -0.7243300529755965,
 -0.2060765850143843,
 -0.13078855261566558,
 0.049335550011889835,
 -0.1748548133546328,
 -0.7150375013187736,
 -0.10213491580370981,
 -0.7741333541615708,
 -0.15224845667319756,
 -0.12861928561336872,
 -0.5305127268803445,
 -0.5198461813168692,
 -0.27740945163335284,
 -0.2012786081718324,
 -0.8241430131259536,
 -0.5267766978117248,
 -0.39226766482160164,
 -0.40479319122724317,
 -0.44790912206827127,
 -0.481646381023405]

In [26]:
# Determine the number of parameters for each model
num_mnl_params = len(display_names)
num_mxl_params = num_mnl_params + len(mixing_variables)

# Get the mle params for the mnl
mnl_results =\
    mnl_obj.fit_mle(np.zeros(num_mnl_params), just_point=True)

# Create the starting values for the mixed logit estimation
mxl_initial_values = np.zeros(num_mxl_params)
mxl_initial_values[:num_mnl_params] = mnl_results['x']

model_obj.fit_mle(mxl_initial_values, num_mxl_draws)
model_obj.get_statsmodels_summary()

Log-likelihood at zero: -3,814.3819
Initial Log-likelihood: -2,835.0507
Estimation Time for Point Estimation: 7.01 minutes.
Final log-likelihood: -2,324.9669


/Users/timothyb0912/Documents/pylogit/pylogit/base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
/Users/timothyb0912/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/timothyb0912/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/timothyb0912/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                3,472
Model:              Mixed Logit Model   Df Residuals:                    3,414
Method:                           MLE   Df Model:                           58
Date:                Fri, 08 Feb 2019   Pseudo R-squ.:                   0.390
Time:                        07:31:32   Pseudo R-bar-squ.:               0.375
AIC:                        4,765.934   Log-Likelihood:             -2,324.967
BIC:                        5,122.778   LL-Null:                    -3,814.382
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
Alternative Specific Constant                                       1.6886      1.725      0.979      0.328      -1.692       5.069
Counseling                                                          1.2051      0.379      3.177      0.001       0.462       1.949
Counseling & medication                                             1.7330      0.451      3.847      0.000       0.850       2.616
Peer support                                                        0.8882      0.376      2.359      0.018       0.150       1.626
Group counseling                                                    0.7778      0.416      1.868      0.062      -0.038       1.594
Early parenting centre programme                                    1.2864      0.328      3.921      0.000       0.643       1.929
Natural, herbal or traditional Chinese medicine                     1.0679      0.384      2.784      0.005       0.316       1.820
Meditation, yoga or exercise                                        1.5746      0.411      3.835      0.000       0.770       2.379
Medication cost                                                    -0.0380      0.004     -9.082      0.000      -0.046      -0.030
Likely to improve                                                   0.5868      0.106      5.515      0.000       0.378       0.795
Home visit                                                          0.0399      0.147      0.272      0.786      -0.248       0.327
Telephone                                                          -0.2657      0.138     -1.923      0.054      -0.536       0.005
Online                                                             -0.0229      0.135     -0.169      0.866      -0.288       0.242
Childcare available                                                 0.2373      0.098      2.419      0.016       0.045       0.430
Age                                                                 0.0113      0.055      0.206      0.837      -0.096       0.119
Employed                                                            0.6385      0.441      1.448      0.148      -0.226       1.503
Unemployed                                                         -2.4623      0.669     -3.678      0.000      -3.774      -1.150
Experience of any treatment type                                    1.1818      0.424      2.789      0.005       0.351       2.012
Low support levels                                                 -1.2325      0.450     -2.736      0.006      -2.115      -0.350
States would seek help                                              0.1866      0.484      0.386      0.700      -0.762       1.135
Income * Cost                                                       0.0001   4.15e-05      2.655      0.008    2.89e-05       0.000
Private health insurance * Cost                                     0.003

# Set checking parameters

In [ ]:
NUM_SAMPLES = 200
RANDOM_SEED = 100

np.random.seed(RANDOM_SEED)

# Sample from the posterior or approximate sampling distribution of model parameters

In [ ]:
class CheckingObject(object):
    def __init__(self, pylogit_obj, num_samples, seed=None):
        # Set object attributes
        self.model = pylogit_obj
        self.hessian = pylogit_obj.hessian
        self.asym_cov = pylogit_obj.cov
        self.asym_dist =\
            scipy.stats.multivariate_normal(
                mean=pylogit_obj.params,
                cov=self.asym_cov,
                allow_singular=True)
        self.posterior_probs = None
        self.sim_y = None
        
        # Set the random seed, if desired
        if seed is not None:
            np.random.seed(seed)

        # Get and set the posterior parameter samples
        self.param_samples = self.asym_dist.rvs(num_samples)
        # Compute and set the posterior probabilities
        self.compute_posterior_probs()
        # Compute and set the simulated choices
        self.simulate_choices()
        return None
        
    def compute_posterior_probs(self):
        self.posterior_probs =\
            self.model.predict(self.model.data,
                               param_list=[self.param_samples.T,
                                           None, None, None])
        return None

    def simulate_choices(self):
        self.sim_y =\
            viz.simulate_choice_vector(
                self.posterior_probs,
                self.model.data[self.model.obs_id_col].values)
        return None

In [ ]:
model_checker =\
    CheckingObject(model_obj, NUM_SAMPLES, seed=RANDOM_SEED)

# Generate posterior predictive datasets

# Save all model generated data

In [ ]:
# Save posterior parameter samples

# Save posterior predictive datasets

# Determine the measurement scales of the variables to be checked 

In [ ]:
# Declare which variables are to be checked
variables_to_check = (explanatory_vars + 
                      ['educ',
                       'hcc',
                       'income',
                       'lact',
                       'married',
                       'post',
                       'preg',
                       'children'
                      ])

In [ ]:
print('Number of unique values per column:')
unique_values_per_variable =\
    clean_df[variables_to_check].agg(
        lambda x: x.unique().size, axis='index')
    
print(unique_values_per_variable)

In [ ]:
# Determine a threshold for which variables will be treated as 
# continuous and which variables will be treated as categorical.
categorical_threshold = 10

# Determine which variables are continuous and which are not
continuous_variables =\
    (unique_values_per_variable[unique_values_per_variable >
                                categorical_threshold]
                               .index.tolist())

categorical_variables =\
    (unique_values_per_variable[unique_values_per_variable <=
                                categorical_threshold]
                               .index.tolist())

# <font color=darkred> Should place all checking related cells in a second notebook.</font>

# Perform the posterior predictive checks

### 1. Predictive Performance plots

In [ ]:
# Generate the simulated log-likelihoods
sim_log_likes =\
    viz.compute_prior_predictive_log_likelihoods(
        model_checker.sim_y,
        clean_df,
        choice_col,
        model_checker.model)

# Plot the simulated versus observed log-likelihood
log_like_path = None
viz.plot_predicted_log_likelihoods(sim_log_likes,
                                   model_checker.model.llf,
                                   output_file=log_like_path)

In [ ]:
# Plot the simulated versus observed log-likelihood for each
# alternative
log_like_path = None
for alt_id in np.sort(clean_df[alt_id_col].unique()):
    alt_idx = clean_df[alt_id_col] == alt_id

    current_sim_y = model_checker.sim_y[alt_idx, :]
    current_obs_y = model_checker.model.choices[alt_idx]

    current_probs =\
        model_checker.model.long_fitted_probs[alt_idx]

    current_sim_log_likes =\
        current_sim_y.T.dot(np.log(current_probs))

    current_log_likelihood =\
        current_obs_y.dot(np.log(current_probs))

    current_alt_label = alt_id_reverse_map[alt_id]

    current_x_label =\
        'Log-Likelihood for {}'.format(current_alt_label)

    viz.plot_predicted_log_likelihoods(current_sim_log_likes,
                                       current_log_likelihood,
                                       x_label=current_x_label,
                                       output_file=log_like_path)

### 2. Outcome Boxplot

In [ ]:
market_path = None
num_obs = model_checker.model.nobs


viz.plot_simulated_market_shares(
    clean_df[alt_id_col].values,
    model_checker.sim_y,
    model_checker.model.choices,
    x_label='Alternative ID',
    y_label='Number\nof times\nchosen',
    output_file=market_path)

### 3. Binned Reliability Plot

In [ ]:
reload(viz)
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    
    current_filter = model_checker.model.alt_IDs == alt
    current_probs =\
        model_checker.model.long_fitted_probs[current_filter]
    current_choices =\
        model_checker.model.choices[current_filter]
    current_sim_y = model_checker.sim_y[current_filter, :]
    
    current_alt = alt_id_reverse_map[alt]
    current_line_label =\
        'Observed vs Predicted ({})'.format(current_alt)
    current_sim_label =\
        'Simulated vs Predicted ({})'.format(current_alt)

    current_sim_color = '#a6bddb'
    current_obs_color = '#045a8d'

    viz.plot_binned_reliability(
        current_probs,
        current_choices,
        sim_y=current_sim_y,
        line_label=current_line_label,
        line_color=current_obs_color,
        sim_label=current_sim_label,
        sim_line_color=current_sim_color,
        figsize=(10, 6),
        ref_line=True,
        output_file=None)

### 4. 'Bagged' Reliability Plot

In [ ]:
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    current_probs =\
        model_checker.model.long_fitted_probs[current_filter]
    current_choices =\
        model_checker.model.choices[current_filter]
    current_sim_y = model_checker.sim_y[current_filter, :]
    
    current_alt = alt_id_reverse_map[alt]
    current_line_label =\
        'Observed vs Predicted ({})'.format(current_alt)
    current_sim_label =\
        'Simulated vs Predicted ({})'.format(current_alt)

    filename = None

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
    fig_and_ax = [fig, ax]
    viz.make_bagged_marginal_model_plot(
        current_sim_y,
        current_choices,
        current_probs,
        y_label=current_line_label,
        prob_label=current_sim_label,
        x_label='Predicted P(Y={})'.format(current_alt),
        alpha=0.5,
        fig_and_ax=fig_and_ax,
        output_file=filename)

    # Determine the maximum value of the x-axis or y-axis
    max_ref_val = max(ax.get_xlim()[1], ax.get_ylim()[1])
    min_ref_val = max(ax.get_xlim()[0], ax.get_ylim()[0])
    # Determine the values to use to plot the reference line
    ref_vals = np.linspace(min_ref_val, max_ref_val, num=100)
    # Plot the reference line as a black dashed line
    ax.plot(ref_vals, ref_vals, 'k--', label='Perfect Calibration')
    ax.legend(loc='best', fontsize=12)
    # Show the plot
    fig.show();

### 5. Binned marginal model plots

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    current_probs = model_checker.posterior_probs[current_filter]
    current_choices =\
        model_checker.model.choices[current_filter]
    current_sim_y = model_checker.sim_y[current_filter, :]
    
    current_alt = alt_id_reverse_map[alt]
    current_line_label =\
        'Observed P(Y={})'.format(current_alt)
    current_sim_label =\
        'Simulated P(Y={})'.format(current_alt)
    current_predicted_label =\
        'Predicted P(Y={})'.format(current_alt)
    for col in continuous_variables:
        current_x = clean_df.loc[current_filter, col].values
    
        viz.make_binned_marginal_model_plot(
            current_probs,
            current_choices,
            current_x,
            partitions=10,
            sim_y=current_sim_y,
            y_label=current_line_label,
            prob_label=current_predicted_label,
            sim_label=current_sim_label,
            x_label=col,
            alpha=0.5,
            figsize=(10, 6),
            output_file=filename)

### 6. Bagged marginal model plots

#### 6a. Check the relationships with the raw explanatory variables

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    current_probs = model_checker.posterior_probs[current_filter]
    current_choices =\
        model_checker.model.choices[current_filter]
    current_sim_y = model_checker.sim_y[current_filter, :]
    
    current_alt = alt_id_reverse_map[alt]
    current_line_label =\
        'Observed P(Y={})'.format(current_alt)
    current_sim_label =\
        'Simulated P(Y={})'.format(current_alt)
    current_predicted_label =\
        'Predicted P(Y={})'.format(current_alt)
    for col in continuous_variables:
        current_x = clean_df.loc[current_filter, col].values

        viz.make_bagged_marginal_model_plot(
            current_probs,
            current_choices,
            current_x,
            sim_y=current_sim_y,
            y_label=current_line_label,
            prob_label=current_predicted_label,
            sim_label=current_sim_label,
            x_label=col,
            alpha=0.5,
            figsize=(10, 6),
            output_file=filename)

#### 6b. Check the relationship with the estimated index, $V = X \beta$

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    
    current_filter = model_checker.model.alt_IDs == alt
    current_probs = model_checker.posterior_probs[current_filter]
    current_choices =\
        model_checker.model.choices[current_filter]
    current_sim_y = model_checker.sim_y[current_filter, :]
    
    current_alt = alt_id_reverse_map[alt]
    current_x =\
        model_checker.model.design.dot(
         model_checker.model.params)[current_filter]
        
    current_y_label = 'Observed P(Y={})'.format(current_alt)
    current_prob_label = 'Predicted P(Y={})'.format(current_alt)
    current_sim_label = 'Simulated P(Y={})'.format(current_alt)

    viz.make_bagged_marginal_model_plot(
        current_probs,
        current_choices,
        current_x,
        sim_y=current_sim_y,
        y_label=current_y_label,
        prob_label=current_prob_label,
        sim_label=current_sim_label,
        x_label=r'$V = X \beta$',
        alpha=0.5,
        figsize=(10, 6),
        fontsize=13,
        output_file=filename)

### 7. Simulated KDEs

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    current_title = 'Y = {}'.format(alt)

    for col in continuous_variables:
        viz.plot_simulated_kde_traces(
            model_checker.sim_y,
            clean_df,
            current_filter,
            col,
            choice_col,
            label='Simulated {}'.format(col),
            title=current_title,
            figsize=(10, 6),
            output_file=filename)

### 8. Simulated CDFs

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    current_title = 'Y = {}'.format(alt)

    for current_col in continuous_variables:
        viz.plot_simulated_cdf_traces(
            model_checker.sim_y,
            clean_df,
            current_filter,
            current_col,
            choice_col,
            label='Simulated ({})'.format(col),
            title=current_title,
            figsize=(10, 6),
            output_file=filename)

### 9. Simulated Histograms

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    alt_name = alt_id_reverse_map[alt]
    current_filter_name = '{} choices'.format(alt_name)
    current_title = ''

    for current_col in categorical_variables:
        viz.plot_categorical_predictive_densities(
            clean_df,
            None,
            model_checker.sim_y,
            current_col,
            current_filter,
            model_checker.model.choices.astype(int),
            title=current_title,
            filter_name=current_filter_name,
            post_color=sbn.color_palette('colorblind')[0],
            figsize=(10, 6),
            legend_loc='best',
            output_file=filename)

###  10. Simulated Lagrange Multiplier tests for mixing

### Generate artificial variables for Lagrange Multiplier Checks 
Note that this is based on the Lagrange Multiplier tests described in McFadden, D., Train, K., 2000. Mixed MNL Models for Discrete Response. Journal of Applied Econometrics 15, 447–470.

In [ ]:
# Get the rows_to_obs matrix
rows_to_obs = model_obj.get_mappings_for_fit()['rows_to_obs']

# Get the mean attribute values for each observation
probs_to_obs = rows_to_obs.multiply(model_obj.long_fitted_probs[:, None])

# Will have shape (num_obs, design[1])
x_mean_per_obs = probs_to_obs.T.dot(model_obj.design)

# Will have same shape as the design matrix
long_x_mean_per_obs = rows_to_obs.dot(x_mean_per_obs)

# X - X_mean_per_obs
augmented_x = model_obj.design - long_x_mean_per_obs

# z = 0.5 * (x - x_bar)^2
artificial_x = 0.5 * augmented_x**2

### Make the desired plots

In [ ]:
filename = None
for alt in np.sort(np.unique(model_checker.model.alt_IDs)):
    current_filter = model_checker.model.alt_IDs == alt
    alt_name = alt_id_reverse_map[alt]
    current_filter_name = '{} choices'.format(alt_name)
    current_title = '' 

    current_probs =\
        model_checker.posterior_probs[current_filter]
    current_choices =\
        model_checker.model.choices[current_filter]
    current_sim_y = model_checker.sim_y[current_filter, :]
    
    current_y_label = 'Observed P(Y={})'.format(alt_name)
    current_prob_label = 'Predicted P(Y={})'.format(alt_name)
    current_sim_label = 'Simulated P(Y={})'.format(alt_name)
    
    for col in range(model_checker.model.design.shape[1]):
        column_name = (model_checker.model
                                          .params
                                          .index
                                          .tolist()[col])
        current_x = artificial_x[current_filter, col]
        current_x_label =\
            'Artificial {} {}'.format(alt_name, column_name)


        viz.make_bagged_marginal_model_plot(
            current_probs,
            current_choices,
            current_x,
            sim_y=current_sim_y,
            y_label=current_y_label,
            prob_label=current_prob_label,
            sim_label=current_sim_label,
            x_label=current_x_label,
            alpha=0.5,
            figsize=(10, 6))

# Findings and Recommendations based on the posterior predictive checks

0. Replicating the published paper was not difficult because the various transformations of the raw data file were not explained in great enough detail, and the raw data is (without transformation), insufficient to replicate the published results. I had to reverse engineer the data transformations, and this took many hours.
1. Based on the predictive performance plots of the log-likelihood, the model for the choice of Alternative B is unrealistic.
2. An alternative specific constant is likely needed for "Alternative A" or "Alternative B".
3. The bagged reliability plots for Alternative N show that the predicted probabilities for this alternative are generally over-estimates once the predicted probabilities are above 0.5.
4. The binned and bagged marginal model plots show that the effects of age and income times cost need to be more flexibly modeled. This is reiterated by the simulated kde and simulated cdf plots.
5. The simulated histograms showed tons of examples, for each alternative, of categorical variables that were not being realistically modeled. The biggest reason I can think of for this is the fact that these categorical variables were specified with an equal effect on the systematic utilities of alternative A and alternative B. I think the simulated histograms are showing that this hypothesis (overall) is likely to be false.
6. The Lagrange Multiplier checks suggests that mixing may be appropriate for some variables. For example: the various interactions between the socio-demographics and the treatment types, experience of any treatment type, low support levels, stating that one would seek help, etc. However, it is unclear how much of this is just an artifact of the systematic utility being underfit (as shown by many of the simulated histograms).
